In [1]:
from observations import mnist
import numpy as np
import tensorflow as tf
import os

/home/maxi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def load_mnist():
    """ loads data from mnist function and converts to numpy arrays"""

    (train_im, train_lab), (test_im, test_lab) = mnist('data/mnist')
    train_im_mean = np.mean(train_im, 0)
    train_im_std = np.std(train_im, 0)

    std_eps = 1e-7
    train_im = np.array((train_im - train_im_mean) / (train_im_std + std_eps))
    test_im = np.array((test_im - train_im_mean) / (train_im_std + std_eps))

    #convert to dictionary
    data = {}
    data['x_train'] = train_im
    data['y_train'] = np.array(train_lab)
    data['x_test'] = test_im
    data['y_test'] = np.array(test_lab)

    return data

#expected = y_test
#predicted = y_test #Network output

#print("Classification report:")
#print(classification_report(expected, predicted))
#print("Confusion matrix:")
#print(confusion_matrix(expected, predicted))

def get_logs_path(session):
    """ Returns path where log is going to be saved.
    @Input session is the parent folder for the runs
    So e.g. (Session1 -> run_00, run_01, run_02 ),
    (Session2 -> run_00, run_01)"""
    curr_dir = os.getcwd()
    logs_path = curr_dir + '/tensorflow_logs/train_mnist/' + session
    if not os.path.exists(logs_path):
        os.makedirs(logs_path)
    previous_runs = os.listdir(logs_path)
    new_path = logs_path + "/run_{:02d}".format(len(previous_runs))
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        print("Saving log at: ", new_path)
        return new_path
    raise Exception("Path is already existing: ", new_path)

In [3]:
""" Ziel ist eine nn struktur die sich durch tweaken von eins zwei parametern komplett verändern lässt
dabei sollte die depth variable sein, so wie wie viele hidden units ich habe
wie die variablen aufgesetzt werden sollte auch per input regulierbar sein
Maybe with dropout etc."""

def forward(input_samples, num_hidden=2, output_units=[20,10,10,10]):
    """ this is the whole network structure, returns the last layer after computing
    every other layer"""
    
    #assert that inputs are vaguely corretct:
    assert len(output_units) == num_hidden + 2,\
    "output_units length: {}, while only {} num_hidden".format(len(output_units),  num_hidden)
    
    #reshape input data
    input_layer = input_samples
    
    #input layer
    first_layer = tf.layers.dense(inputs=input_layer, units=output_units[0], name="Input_layer")
    
    layers = {
        "l0": first_layer 
    }
    
    #some deep layers num_hidden iterations, just shifted one forward
    for i in range(1,num_hidden+1):
        prev_layer = layers["l"+str(i-1)]
        layers["l"+str(i)] = tf.layers.dense(inputs=prev_layer, 
                                             units=output_units[i], name="Hidden_layer_"+str(i))
        
    
    #output layer
    out_layer = tf.layers.dense(inputs=layers["l"+str(num_hidden)], units=output_units[-1], name="Output_Layer")
    return out_layer

In [4]:
def train(x_train, y_train, batch_size, input_size, num_epochs, writer):
    """ trains the network defined in forward """
    
    x_ph = tf.placeholder(tf.float32, [batch_size, input_size], name="x_ph")
    y_ph = tf.placeholder(tf.int32, [batch_size], name="y_ph")
    
    print("x placeholder: ", x_ph)
    prediction = forward(x_ph)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels= y_ph, logits=prediction)
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss)
    
    tf.summary.scalar("loss",loss)
    summ = tf.summary.merge_all()
    
    batches_per_epoch = x_train.shape[0] // batch_size
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(num_epochs):
            for batch in range(batches_per_epoch):
                x_batch = x_train[batch * batch_size : (batch+1) * batch_size]
                y_batch = y_train[batch * batch_size : (batch+1) * batch_size]
                
                print("shape of input", y_batch.shape)
                
                _, cur_loss, cur_summ = sess.run([train_op, loss, summ], feed_dict={x_ph: x_batch, y_ph: y_batch})
                writer.add_summary(cur_summ, epoch*batches_per_epoch + batch)
                print(cur_loss)

In [5]:
def run_model():
    x_train, y_train, x_test, y_test = get_mnist(1000,300)
    logs_path = get_logs_path("Session_00")
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    train(x_train=x_train, y_train=y_train, batch_size=100, input_size=784, num_epochs=5, writer=writer)

In [8]:
run_model()

ValueError: Expected 2D array, got 1D array instead:
array=[1. 7. 9. 2. 1. 1. 5. 9. 9. 9. 3. 6. 9. 2. 1. 6. 1. 0. 7. 0. 8. 0. 8. 2.
 7. 1. 6. 5. 0. 7. 3. 2. 9. 1. 5. 7. 9. 3. 0. 0. 8. 5. 2. 3. 7. 2. 7. 3.
 6. 3. 9. 5. 6. 0. 8. 8. 7. 7. 8. 2. 7. 0. 2. 6. 4. 6. 2. 1. 3. 9. 0. 4.
 8. 4. 3. 9. 5. 9. 0. 3. 2. 4. 5. 7. 2. 9. 9. 8. 3. 2. 5. 4. 3. 4. 0. 2.
 9. 9. 7. 4. 8. 2. 6. 9. 4. 3. 7. 1. 8. 0. 0. 8. 2. 5. 8. 4. 5. 7. 6. 1.
 5. 9. 1. 4. 5. 4. 9. 9. 6. 8. 7. 8. 6. 3. 6. 0. 7. 5. 1. 3. 7. 1. 3. 4.
 7. 8. 6. 7. 8. 2. 0. 2. 7. 6. 2. 9. 5. 9. 1. 3. 5. 2. 2. 4. 8. 1. 6. 3.
 0. 4. 3. 7. 8. 6. 9. 0. 6. 9. 5. 4. 2. 5. 6. 5. 0. 3. 1. 1. 5. 3. 6. 3.
 4. 0. 1. 5. 8. 5. 6. 4. 9. 3. 3. 3. 1. 4. 3. 0. 8. 9. 0. 3. 7. 5. 9. 7.
 2. 5. 7. 9. 2. 6. 6. 3. 9. 7. 6. 1. 9. 7. 0. 7. 4. 9. 8. 2. 2. 5. 9. 6.
 2. 6. 0. 1. 1. 8. 9. 2. 9. 2. 2. 2. 3. 7. 0. 9. 5. 9. 4. 7. 9. 3. 3. 9.
 8. 4. 5. 4. 7. 6. 3. 9. 1. 0. 6. 3. 1. 3. 0. 2. 4. 1. 5. 7. 4. 4. 7. 1.
 4. 2. 7. 9. 9. 9. 9. 7. 0. 2. 1. 8. 6. 7. 2. 9. 1. 2. 1. 1. 9. 2